# 🐈 Cat Discriminator

### 📝 Imports

In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import albumentations as A

import torchvision
import torchvision.transforms as transforms

import os

from PIL import Image

from cat_discriminator_neural_net import CatDiscriminatorNeuralNet

from src.augmentation.data_augmenter import DataAugmenter


c:\Users\Dan\AppData\Local\Programs\Python\Python312\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


### 🔧 Config

In [2]:
image_size = 512

learning_rate = 0.01
epochs_between_saves = 5
total_epochs = 200

saved_model_path = "trained_networks/cat_discriminator.pth"

training_data_path = 'data/staging'

bathroom_cat_path = '/bathroom-cat'
captain_dir_path = '/captain'
control_dir_path= '/control'

### 🌐 Create Transforms

In [3]:
transforms = transforms.Compose([
    DataAugmenter(image_size),
    transforms.ToTensor(),
])

DataAugmenter initialized


### 🚦 Load Training Data

In [4]:
from cats_dataset import CatsDataset


dataset = CatsDataset(
    bathroom_cat_dir=training_data_path + bathroom_cat_path, 
    captain_dir=training_data_path + captain_dir_path, 
    control_dir=training_data_path + control_dir_path, 
    transform=transforms)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)

### 🥾 Initialize the Neural Net

In [5]:
net = CatDiscriminatorNeuralNet()

if os.path.isfile(saved_model_path):
    net.load_state_dict(torch.load(saved_model_path))

net.cuda()
loss_function = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate)

C:\Users\Dan\AppData\Local\Temp\ipykernel_13784\412691464.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(saved_model_path))


### 🏃‍♂️‍➡️ Train

In [6]:
epoch = 1
while epoch <= total_epochs:
    print(f'Training epoch {epoch}')
    
    running_loss = 0.0
    for i, data in enumerate(dataloader):
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()

        outputs = net(inputs)
        
        loss = loss_function(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        
    if epoch % epochs_between_saves == 0:
        print('Saving model...')
        torch.save(net.state_dict(), saved_model_path)
        
    print (f'Loss: {running_loss / len(dataloader):.4f}')
    epoch += 1

Training epoch 1
Loss: 0.3562
Training epoch 2
Loss: 0.3644
Training epoch 3
Loss: 0.3536
Training epoch 4
Loss: 0.3038
Training epoch 5


KeyboardInterrupt: 

### 💾 Save Progress

In [7]:
torch.save(net.state_dict(), saved_model_path)